ECS308: Data Science and Machine Learning Project | Akanksha Singh, 19022
    

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Loading the data
df = pd.read_csv('dataset.csv')
df

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,2015825,2508182,HH781001,11/15/2002 12:30:00 PM,003XX N ASHLAND AVE,0810,THEFT,OVER $500,STREET,False,...,27.0,28.0,06,1165667.0,1902638.0,2002,02/28/2018 03:56:25 PM,41.888430,-87.667064,"(41.888429628, -87.667063831)"
1,2110300,2687361,HJ308830,04/18/2003 06:10:00 PM,016XX W 18TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,...,25.0,31.0,08B,1165650.0,1891466.0,2003,02/10/2018 03:50:01 PM,41.857773,-87.667445,"(41.857773091, -87.667444544)"
2,381828,10755691,HZ518320,11/16/2016 09:50:00 PM,009XX S CICERO AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,True,...,24.0,25.0,16,1144531.0,1895566.0,2016,02/10/2018 03:50:01 PM,41.869447,-87.744861,"(41.869447309, -87.744861466)"
3,6048052,8740111,HV415603,08/04/2012 05:15:00 PM,072XX S INGLESIDE AVE,0460,BATTERY,SIMPLE,OTHER,False,...,5.0,69.0,08B,1183801.0,1857416.0,2012,02/04/2016 06:33:39 AM,41.763932,-87.601884,"(41.763931739, -87.601884055)"
4,6873402,11924187,JC549979,12/16/2019 04:45:00 PM,017XX W THOME AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,...,40.0,77.0,14,1163394.0,1941667.0,2019,12/23/2019 03:56:38 PM,41.995575,-87.674309,"(41.995575459, -87.674309158)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3562445,4714376,HM318809,04/21/2006 12:01:00 AM,022XX S WENTWORTH AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESTAURANT,False,...,25.0,34.0,14,1175327.0,1889358.0,2006,02/10/2018 03:50:01 PM,41.851777,-87.631988,"(41.851777151, -87.631987613)"
99996,4621702,6567536,HP637109,10/19/2008 10:05:00 PM,002XX W CERMAK RD,1360,CRIMINAL TRESPASS,TO VEHICLE,ALLEY,True,...,25.0,34.0,26,1175002.0,1889741.0,2008,02/28/2018 03:56:25 PM,41.852835,-87.633169,"(41.852835409, -87.633168974)"
99997,5483697,7815219,HS625058,11/20/2010 04:50:00 PM,0000X N STATE ST,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,True,...,42.0,32.0,06,1176362.0,1900577.0,2010,02/10/2018 03:50:01 PM,41.882540,-87.627851,"(41.882539542, -87.627850543)"
99998,3616848,4921843,HM536277,08/12/2006 05:29:00 PM,032XX W ROOSEVELT RD,0530,ASSAULT,AGGRAVATED: OTHER DANG WEAPON,ALLEY,True,...,24.0,29.0,04A,1154766.0,1894523.0,2006,02/28/2018 03:56:25 PM,41.866387,-87.707314,"(41.866386519, -87.707313676)"


In [3]:
df = df.drop(['Latitude', 'Longitude'], axis = 1) #redundant feature, already used in location

In [14]:
df = df.astype({"Unnamed: 0": object})
df = df.astype({"ID": object})
df = df.astype({"Beat": object})
df = df.astype({"District": object})
df = df.astype({"Ward": object})
df = df.astype({"Community Area": object})
df = df.astype({"Year": object})
df = df.astype({"X Coordinate": object})
df = df.astype({"Y Coordinate": object})

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100000 entries, 2002-11-15 12:30:00 to 2008-07-30 22:00:00
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Unnamed: 0            100000 non-null  object        
 1   ID                    100000 non-null  object        
 2   Case Number           100000 non-null  object        
 3   Date                  100000 non-null  datetime64[ns]
 4   Block                 100000 non-null  object        
 5   IUCR                  100000 non-null  object        
 6   Primary Type          100000 non-null  object        
 7   Description           100000 non-null  object        
 8   Location Description  100000 non-null  object        
 9   Arrest                100000 non-null  bool          
 10  Domestic              100000 non-null  bool          
 11  Beat                  100000 non-null  object        
 12  District              10

# 3.1. Feature Selection

## 3.1.1 Entropy

In [17]:
def entropy(y):
    probs = [] # Probabilities of each class label
    for c in set(y): # Set gets a unique set of values. We're iterating over each value
        num_same_class = sum(y == c)  # Remember that true == 1, so we can sum.
        p = num_same_class / len(y) # Probability of this class label
        probs.append(p)
    return sum(-p * np.log2(p) for p in probs)

In [18]:
columns = ['Unnamed: 0', 'ID', 'Case Number', 'Date', 'Block', 'IUCR',
       'Primary Type', 'Description', 'Location Description', 'Arrest',
       'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code',
       'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Location']

"""This code takes too long to execute. So, the result has been appended as a column "Entropy" in df_ent

#Code:
for column in columns:
    print(entropy(df[column]))
    df_ent.append(entropy(df[column]))

"""

'This code takes too long to execute. So, the result has been appended as a column "Entropy" in df_ent\n\n#Code:\nfor column in columns:\n    print(entropy(df[column]))\n    df_ent.append(entropy(df[column]))\n\n'

In [19]:
df_ent = pd.DataFrame()
df_ent["Features"] = columns
df_ent["Entropy"] = [16.60964047441802, 16.60964047441802, 16.60964047441802, 16.50325437354276, 
                     13.842338152132987, 5.6494058088293935, 3.482725829513007, 5.518046958190912,4.0556159613837135,
                     0.839130421235637,0.5773624295265078,8.0915039193288,4.403738721836829,5.475453895970417,
                     5.826020205215316,3.5574293236120447,14.760131667723728,15.11747552923421,4.239196753831807,
                     2.8553923756999566,15.74133241017893]

df_ent

,Features,Entropy
0,Unnamed: 0,16.609640
1,ID,16.609640
2,Case Number,16.609640
3,Date,16.503254
4,Block,13.842338
5,IUCR,5.649406
6,Primary Type,3.482726
7,Description,5.518047
8,Location Description,4.055616
9,Arrest,0.839130


In [20]:
#Dropping high entropy features(Entropy > 10)

df_drop = df.drop(columns =['Unnamed: 0', 'ID', 'Case Number', 'Date', 
                        'Block', 'X Coordinate', 'Y Coordinate', 'Location'])

df_drop.reset_index(drop = True)

,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Updated On
0,0810,THEFT,OVER $500,STREET,False,False,1333,12.0,27.0,28.0,06,2002,02/28/2018 03:56:25 PM
1,0486,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,True,1222,12.0,25.0,31.0,08B,2003,02/10/2018 03:50:01 PM
2,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,True,False,1131,11.0,24.0,25.0,16,2016,02/10/2018 03:50:01 PM
3,0460,BATTERY,SIMPLE,OTHER,False,False,324,3.0,5.0,69.0,08B,2012,02/04/2016 06:33:39 AM
4,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,2433,24.0,40.0,77.0,14,2019,12/23/2019 03:56:38 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1310,CRIMINAL DAMAGE,TO PROPERTY,RESTAURANT,False,False,2111,9.0,25.0,34.0,14,2006,02/10/2018 03:50:01 PM
99996,1360,CRIMINAL TRESPASS,TO VEHICLE,ALLEY,True,False,2111,9.0,25.0,34.0,26,2008,02/28/2018 03:56:25 PM
99997,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,True,False,122,1.0,42.0,32.0,06,2010,02/10/2018 03:50:01 PM
99998,0530,ASSAULT,AGGRAVATED: OTHER DANG WEAPON,ALLEY,True,False,1022,10.0,24.0,29.0,04A,2006,02/28/2018 03:56:25 PM


# Part 1 - 100,000 Rows

# Part 2 - 25,000 Rows

We are considering 25k data points due to technical limitations

## 3.2. Feature Engineering

In [30]:
#These data points have unique primary crime type, later we will need to drop it for splitting

df_drop.drop([df_drop.index[64744], df_drop.index[94033]], axis=0, inplace=True)

### 3.2.1. Splitting Data

In [31]:
# splitting data into 25k

from sklearn.model_selection import train_test_split

X = df_drop[['IUCR','Description','Location Description','Arrest','Domestic','Beat','District',
          'Ward','Community Area','FBI Code','Year','Updated On']]
Y = df_drop['Primary Type']

data25k, data75k, test25k, test75k= train_test_split(X, Y, test_size = 0.75, random_state = 50)

data25 = pd.concat([data25k, test25k], axis=1)
data25 = data25.reset_index(drop= True)

data25.to_csv("data25.csv")

### 3.2.2. One Hot Encoding

In [32]:
column_names = []
for row in df_drop:
  column_names.append(row)

column_names

['IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'Year',
 'Updated On']

In [33]:
# generate binary values using get_dummies

df_dum = pd.get_dummies(data25, columns = column_names)
df_dum = df_dum.reset_index(drop = True)

C:\Users\akank\AppData\Local\Temp\ipykernel_15648\1443840504.py:3: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_dum = pd.get_dummies(data25, columns = column_names)
